#### Helper Functions

Pydantic provides three `classmethod` helper functions on models for parsing data:

* `parse_obj`: this is very similar to the `__init__` method of the model, except it takes a dict rather than keyword arguments. If the object passed is not a dict a `ValidationError` will be raised.

* `parse_raw`: this takes a str or bytes and parses it as json, then passes the result to `parse_obj`. Parsing pickle data is also supported by setting the `content_type` argument appropriately.

* `parse_file`: this takes in a file path, reads the file and passes the contents to `parse_raw`. If `content_type` is omitted, it is inferred from the file's extension.

In [1]:
import pickle
from datetime import datetime
from pathlib import Path

from pydantic import BaseModel, ValidationError

In [2]:
class User(BaseModel):
    id: int
    name = "John Doe"
    signup_ts: datetime = None

In [3]:
m = User.parse_obj({"id": 123, "name": "James"})
print(m)

id=123 signup_ts=None name='James'


In [4]:
try:
    print(User.parse_obj(["not", "a", "dict"]))
except ValidationError as e:
    print(e)

1 validation error for User
__root__
  User expected dict not list (type=type_error)


In [5]:
m = User.parse_raw('{"id": 123, "name": "James"}')
print(m)

id=123 signup_ts=None name='James'


In [6]:
pickle_data = pickle.dumps({
    "id": 123,
    "name": "James",
    "signup_ts": datetime(2017, 7, 14),
})
m = User.parse_raw(
    pickle_data, content_type="application/pickle", allow_pickle=True,
)
print(m)

id=123 signup_ts=datetime.datetime(2017, 7, 14, 0, 0) name='James'


In [7]:
path = Path("data.json")
path.write_text('{"id": 123, "name": "James"}')
m = User.parse_file(path)
print(m)

id=123 signup_ts=None name='James'


> ##### Warning
> 
> To quote the official `pickle` docs, "The pickle module is not secure against erroneous or maliciously constructed data. Never unpickle data received from an untrusted or unauthenticated source."

> ##### Info
> 
> Because it can result in arbitrary code execution, as a security measure, you need to explicitly pass allow_pickle to the parsing function in order to load pickle data.

##### Creating models without validation

pydantic also provides the `construct()` method which allows models to be created without validation this can be useful when data has already been validated or comes from a trusted source and you want to create a model as efficiently as possible (`construct()` is generally around 30x faster than creating a model with full validation).

> ##### Warning
> 
> construct() does not do any validation, meaning it can create models which are invalid. You should only ever use the construct() method with data which has already been validated, or you trust.

In [8]:
class NoValidationUser(BaseModel):
    id: int
    age: int
    name: str = "John Doe"

In [9]:
original_user = NoValidationUser(id=123, age=32)
user_data = original_user.dict()
print(user_data)
fields_set = original_user.__fields_set__
print(fields_set)

{'id': 123, 'age': 32, 'name': 'John Doe'}
{'id', 'age'}


In [10]:
new_user = NoValidationUser.construct(_fields_set=fields_set, **user_data)
print(repr(new_user))
print(new_user.__fields_set__)

NoValidationUser(id=123, age=32, name='John Doe')
{'id', 'age'}


In [11]:
bad_user = NoValidationUser.construct(id='dog')
print(repr(bad_user))

NoValidationUser(id='dog', name='John Doe')


The `_fields_set` keyword argument to `construct()` is optional, but allows you to be more precise about which fields were originally set and which weren't. If it's omitted `__fields_set__` will just be the keys of the data provided.

For example, in the example above, if `_fields_set` was not provided, `new_user.__fields_set__` would be `{"id", "age", "name"}`.